# Chapter 09. 로지스틱 회귀의 기초
## 2. 분류 문제의 성능지표
### 01 - 분류 문제에 있어서 성능지표의 필요성
- 로지스틱 회귀는 대표적인 분류 문제를 위한 알고리즘

- 기본적으로 우리가 다루는 여러 문제마다 회귀, 분류, 클러스터링 등의 다양한 성능지표들이 존재
    - **회귀(regression)** : MAE, MSE, RMSE, SSE
    - **분류(classification)** : 정확도, 정밀도, 민감도, F1 스코어, ROC 커브, 리프트 차트
    - **클러스터링(clustering)** : DBI, 엘보우 메서드, 실루엣계수

- 머신러닝에서 다양한 추가 성능지표를 사용하는 예
    - 위에 말한 기본 성능지표 외에도 실제 머신러닝을 적용한 시스템에서 사용가능한 다양한 추가 성능지표들이 있음
    - 이 때 다음과 같은 여러 상황을 고려하여 모델의 성능지표를 선택해야 함
        - 1) 모델이 다른 모델보다 경제적으로 나은가?
        - 2) 모델이 사용하는 데이터가 많은가? or 적은가?
        - 3) 모델이 용량이 작은 컴퓨터에서도 작동하는가?
        - 4) 모델의 손해가 얼마나 나는가?

### 02 - 혼동행렬
- 분류 문제의 성능 측정을 위해 먼저 이해해야 할 것은 혼동행렬

- **혼동행렬(confusion matrix)** : 예측값이 실제값 대비 얼마나 잘 맞는지 $2 \times 2$ 행렬로 표현하는 기법
    - 실제값(actual class)은 수식에서 $y$, 예측값(prediction)은 모델이 예측하는 $\hat {y}$
        - 이 두 가지 값을 바탕으로 총 4가지 결과가 나타남
    - 여기서 기억해야 할 것은 1과 0값의 의미
        - 모델의 $y$값을 어떻게 구성하는지에 따라 다르지만 일반적으로 질문의 '예'에 해당하는 값은 True 또는 1, '아니오'에 해당하는 값은 False 또는 0

- 실제값과 예측값의 조합으로 발생 가능한 경우는 총 4가지
    - 예측값과 실제값이 동일한 경우, 즉 모델의 예측이 정답일 때 True를 붙임
    - 예측값이 1을 예측했다면 Positive, 0을 예측했다면 Negative
    - 1) **True Positive(TP)** : 예측값과 실제값이 모두 1로 동일할 때
		- 모델의 예측값이 정답이고 예측 대상이 1일 때
    - 2) **True Negative(TN)** : 예측값과 실제값이 모두 0으로 동일할 때
		- 모델의 예측값이 정답이고 예측 대상이 0일 때
    - 3) **False Negative(FN)** : 실제값은 1이지만 예측값이 0일 때
		- 모델의 예측값이 오답이고 예측값이 0을 예측할 때
    - 4) **False Positive(FP)** : 실제값은 0이지만 예측값이 1일 때
		- 모델의 예측값이 오답이고 예측값이 1을 예측할 때

- 혼동행렬표는 간단히 사이킷런을 사용하여 나타낼 수 있음
    - sklearn.metrics 모듈 안에 confusion_matrix 함수로 행렬 표시
    - 이 때 입력되는 값은 y_true, y_pred

In [1]:
from sklearn.metrics import confusion_matrix
y_true = [1, 0, 1, 1, 0, 1]
y_pred = [0, 0, 1, 1, 0, 1]
# 지금은 y_pred에 임의의 값을 입력했지만 실제로는 모델의 예측값이 들어감
confusion_matrix(y_true, y_pred)

array([[2, 0],
       [1, 3]], dtype=int64)

In [2]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
(tn, fp, fn, tp)

(2, 0, 1, 3)

### 03 - 혼동행렬표를 사용한 지표
- 혼동행렬을 바탕으로 이진분류를 위한 다양한 지표들 개발 가능
    - 단순히 얼마나 맞췄는지를 측정하는 정확도(accuracy)
    - 1이라고 예측했을 때 그 값이 얼마나 정답이었는지를 보여주는 정밀도(precision)
    - 1인 값을 얼마나 안 놓치고 정답으로 맞추는지에 대한 민감도(recall)

- 이러한 지표들이 정확히 어떻게 사용되는지에 대해서 이해하고 사용하는 것이 이진분류 문제의 성능 측정을 위해 매우 중요

#### 3.1 정확도
- **정확도(accuracy)** : 전체 데이터 개수 대비 정답을 맞춘 데이터의 개수
    - $Accuracy = \frac{TP + TN}{TP + TN + FP + FN}$

- sklearn.metrics 모듈 안에 있는 **accuracy_score** 사용

In [3]:
import numpy as np
from sklearn.metrics import accuracy_score

y_pred = np.array([0, 1, 1, 0])
y_true = np.array([0, 1, 0, 0])

sum(y_true == y_pred) / len(y_true), accuracy_score(y_true, y_pred)

(0.75, 0.75)

#### 3.2 정밀도, 민감도, F1 스코어
- 정밀도와 민감도는 비슷하지만 조금 다름
    - 불균일한 데이터셋을 다룰 때 유용한 지표
        - 불균일한 데이터셋 : 데이터에서 1과 0의 비율이 7:3 or 3:7 이상으로 차이나는 데이터셋

- **정밀도(precision)** : 모델이 1이라 예측했을 때 얼마나 잘 맞을지에 대한 비율
    - 분모는 모델이 1이라고 예측한 데이터의 개수, 분자는 그 중 정답의 개수
    - $PRECISION(PPV) = \frac{TP}{TP + FP}$

- **민감도(recall)** : 실제 1인 값을 가진 데이터를 모델이 얼마나 1이라고 잘 예측했는지에 대한 비율
    - 반환율 또는 재현율이라고도 함
    - 분모는 실제값이 1인 데이터의 개수, 분자는 모델이 맞춘 데이터의 개수
    - $RECALL(TPR) = \frac{TP}{TP + FN}$

- **F1 스코어(F1 score)** : 정밀도와 민감도를 통합한 지표, 정밀도와 민감도의 조화평균 값
    - $F_1 = 2 * \frac{precision \times recall}{precision + recall}$

- sklearn.metrics 모듈 안에 있는 **precision_score, recall_score, f1_score** 사용

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score

y_pred = np.array([0, 1, 1, 0, 1, 1, 1, 0])
y_true = np.array([0, 1, 0, 0, 0, 0, 1, 1])

precision_score(y_true, y_pred)
# 정밀도

0.4

In [6]:
recall_score(y_true, y_pred)
# 민감도

0.6666666666666666

In [7]:
f1_score(y_true, y_pred)
# F1 스코어

0.5